In [ ]:
import os
import sys 

while "src" not in os.listdir(): os.chdir("..")

if "./src" not in sys.path: sys.path.append("./src")

In [ ]:
from os import getenv

from uuid import uuid4

from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
  api_key=getenv("OPENAI_API_KEY"),
  base_url="https://openrouter.ai/api/v1",
  model="gpt-4o-mini",
)

In [ ]:
import os
import openai

from typing import TypedDict

from langgraph.checkpoint.memory import InMemorySaver

from langgraph.graph import StateGraph, START, END
from langgraph.types import interrupt, Command

openai.api_key = os.environ.get("OPENAI_API_KEY")

class SubAgentState(TypedDict):
    user_input: str = ""
    ai_output: str = ""
    complete:bool = False 

def input_node(state: SubAgentState):
    """
    Node 1: interrupt for input and store it in StateGraph['user_input'].
    (In a real langgraph node this would be an interrupt node; here we use Python input().)
    """
    prompt = interrupt("Enter prompt (or 'exit' to exit): ").strip()
    if isinstance(prompt, dict):
        state = prompt["state"]
        prompt = prompt["prompt"]
    state['user_input'] = prompt 
    state["complete"] = state["user_input"].strip().lower() == "exit"
    return state 

def llm_node(state: SubAgentState):
    """
    Node 2: call LLM using the stored input in StateGraph['user_input'].
    After running, this node returns control back to the input node (the loop below).
    """
    ai_message = llm.invoke(state["user_input"])
    state["ai_output"] = ai_message.content
    print (state["ai_output"])
    return state

def quit_or_not(state: SubAgentState):
    return "llm_node" if not state["complete"] else END

builder = StateGraph(SubAgentState)
builder.add_node("input_node", input_node)
builder.add_node("llm_node", llm_node)

builder.add_edge(START, "input_node")
builder.add_conditional_edges("input_node", quit_or_not, ["llm_node", END])
builder.add_edge("llm_node", "input_node")

config = {"configurable": {"thread_id": str(uuid4())}}
sa = builder.compile(checkpointer=InMemorySaver())

# interrupted = False 
# result = {"user_input": "", "ai_output": "", "complete": False}
# while not result["complete"]: 
#     if interrupted:
#         result = sa.invoke(Command(resume=input("Prompt: ")), config=config)
#     else:
#         result = sa.invoke(input=result, config=config)
#     interrupted = "__interrupt__" in result
        
    

In [ ]:
import json 
from uuid import uuid4
from typing import Annotated, Literal, List, Dict, TypedDict

from langchain.tools import ToolRuntime, tool
from langchain_core.messages import HumanMessage, AnyMessage, SystemMessage, ToolMessage, AIMessage
from langgraph.types import Command, interrupt, Send
from langgraph.graph import add_messages

from langgraph.prebuilt import ToolNode



In [ ]:
import json 
from uuid import uuid4
from typing import Annotated
from langchain.tools import InjectedToolCallId
# from langchain_core.tools import tool
from langchain.tools import ToolRuntime, tool
from langchain_core.messages import HumanMessage, AnyMessage, SystemMessage, ToolMessage
from langgraph.types import Command, interrupt, Send
from langgraph.graph import add_messages
from typing import Literal

from langgraph.prebuilt import ToolNode

# We will assume there will be only one agent tool call

@tool
def chatting_agent_tool(query:str, runtime: ToolRuntime):
    """A tool for interacting with a specialized agent."""

    _state = runtime.state.get("chatting_agent_tool_state", {})
    _complete = _state.get("complete", False)
    result = {"user_input": "query", "ai_output": "", "complete": _complete}

    _interrupted = _state.get("interrupted", False)
    if _interrupted: 
        result = Command(resume = _state.get("resume_content", {}))
    
    result = sa.invoke(result, config=runtime.config)
    
    if "__interrupt__" in result.keys():
        return Command(
                goto=Send("interrupt_node", arg={"next_node": "tool_executor", "__interrupt__": result["__interrupt__"]})
                update={
                        ""
                    }
            )
    else:
        return Command(update={
                "messages": result["ai_output"], 
                "chatting_agent_tool_state": {
                        "interrupted": False, 
                        "interrupt_content": {}, 
                        "resume_content": {}
                    }
            }) 
    
class InterruptNodeState(TypedDict):
    where_to_next: str

class MasterState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    chatting_agent_tool_state: ChattingAgentToolState
    interrupt_node_state: InterruptNodeState

tools = [chatting_agent_tool] 
tool_name_to_executables = {tool.name: tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)

def llm_call(state: MasterState) -> MasterState:

    _input = [SystemMessage("You are a routing agent. Route the user to chat with the specialized agent.")] + \
                state["messages"]

    ai_message = llm_with_tools.invoke(
            input=_input
        )
    state["messages"] = [ai_message]
    return state

def check_tool_call(state: MasterState) -> Literal["tool_executor", "END"]:
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tool_executor"
    else:
        return END
    
def interrupt_node(state: MasterState) -> MasterState:
    

builder = StateGraph(MasterState)
builder.add_node("llm_call", llm_call)
builder.add_node("tool_executor", ToolNode(tools=[chatting_agent_tool]))
builder.add_edge(START, "llm_call")
builder.add_conditional_edges("llm_call", check_tool_call, ["tool_executor", END])
builder.add_edge("tool_executor", "llm_call")

ma = builder.compile(checkpointer=InMemorySaver())

blar = True
config = {"configurable": {"thread_id": str(uuid4())}}
interrupted = False 
result = {"messages": [], "interrupted": False}
inputs = ["testing", "exit"]
input_iter = iter(inputs)
while blar: 
    if interrupted:
        prompt = next(input_iter) # input("Prompt:")
        resume = {
            "prompt": prompt, 
            "config": config,
            "state": result,
        }
        result = ma.invoke(Command(resume=resume, update={"interrupted" : True}), config=config)
    else:
        result = ma.invoke(input=result, config=config)
    interrupted = "__interrupt__" in result


In [ ]:
from langchain.tools import tool, InjectedToolArg, InjectedToolCallId, ToolRuntime
from langgraph.prebuilt import ToolNode, InjectedState, tools_condition
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.types import Command, interrupt
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, BaseMessage, SystemMessage
from typing import Annotated
from copy import deepcopy
from langgraph.runtime import get_runtime

@tool
def test(query: str, x: Annotated[str, InjectedToolArg], tool_call_id: Annotated[str, InjectedToolCallId], runtime: ToolRuntime) -> int:
    """Test"""
    print (runtime.state)
    return ToolMessage(
            content="",
            artifact={"query": query, "x": x},
            tool_call_id = tool_call_id
        )


# AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_JSw0YCATU7F215AaxyfNk0vq', 'function': {'arguments': '{"a": 2, "b": 3}', 'name': 'multiply'}, 'type': 'function'}, {'index': 1, 'id': 'call_AVz2aq3nH2CfS9ZvlvYxYeNB', 'function': {'arguments': '{"a": 6, "b": 10}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0'}, id='run--bec3fe12-cdbf-4b6b-ba7b-457ff110f1d9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_JSw0YCATU7F215AaxyfNk0vq', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 6, 'b': 10}, 'id': 'call_AVz2aq3nH2CfS9ZvlvYxYeNB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 76, 'output_tokens': 51, 'total_tokens': 127, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),

tool_calls = [{'index': 0, 'id': 'call_JSw0YCATU7F215AaxyfNk0vq', 'function': {'arguments': '{"query": "haha", "x": "he"}',  'name': 'test'}, 'type': 'function'}]
graph = StateGraph(MessagesState).add_node("tool", ToolNode(tools=[test])).add_edge(START, "tool").add_edge("tool", END).compile()
ai_message = AIMessage(content='', additional_kwargs={'tool_calls': tool_calls})


graph.invoke({"messages": [ai_message]})
